# Update `treasury_rates_df.pkl`
Last updated by Developer on 2024-01-09.

This notebook allows one to inspect and manually update the `treasury_rates_df.pkl` file that is used in the production. This is particularly helpful if the cloud function `daily_treasury_yield` running on cloud scheduler `daily_treasury_rate` fails. The `date` variable is a string representation of the date that needs to be added into the dataframe, and `new_values` represents a dictionary with keys corresponding to the year and the corresponding treasury rate value. Note: the order of the values is important in `new_values` as the keys are only for user guidance, as only the values of the dictionary are used. Note, if the cloud function is run with a different version of pandas, update the pandas version in the cloud function and re-run it.

In [ ]:
from datetime import datetime

from google.cloud import storage

In [ ]:
import pandas as pd
assert pd.__version__ == '1.3.5'    # NOTE: pandas version needs to be consistent with the one that is on the server (deployed from `mitas-server`). Currently that version is '1.3.5'

In [ ]:
import os
import sys


__file__ = os.path.abspath('update_treasury_rates_df.ipynb')    # in a Jupyter Notebook, the `__file__` variable is not automatically defined because notebooks do not run as standard Python scripts
server_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', 'app_engine', 'demo', 'server'))    # get the directory containing the 'app_engine/demo/server' package
sys.path.append(server_dir)    # add the directory to sys.path


from modules.get_creds import get_creds
get_creds()


from modules.ficc.utils.gcp_storage_functions import download_pickle_file, upload_data

In [ ]:
storage_client = storage.Client()

In [ ]:
bucket_name = 'treasury_rate_df'
file_name = 'treasury_rate_df.pkl'

In [ ]:
treasury_rate_df = download_pickle_file(storage_client, bucket_name, file_name)    # create function instead of loading as a global variable so we can get the most updated value every time we price a CUSIP; updated daily by the cloud function `daily_treasury_yield`
treasury_rate_df

In [ ]:
date = '01-08-2024'
date = datetime.strptime(date, '%m-%d-%Y').date()

In [ ]:
new_values = {'year_1': 4.82, 
              'year_2': 4.36, 
              'year_3': 4.11, 
              'year_5': 3.97, 
              'year_7': 3.99, 
              'year_10': 4.01, 
              'year_20': 4.33, 
              'year_30': 4.17}

In [ ]:
treasury_rate_df.loc[date] = new_values.values()
treasury_rate_df = treasury_rate_df.sort_index(ascending=False)
treasury_rate_df[~treasury_rate_df.index.duplicated(keep='first')]
treasury_rate_df

In [ ]:
tmp_file_name = f'/tmp/{file_name}'
treasury_rate_df.to_pickle(tmp_file_name)
upload_data(storage_client, bucket_name, file_name, tmp_file_name)